<a href="https://colab.research.google.com/github/Kalloniatis/Humour_Recognition_in_Greek/blob/main/WordEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download el_core_news_lg

2023-10-29 06:48:07.618241: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-29 06:48:07.618306: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-29 06:48:07.618358: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-29 06:48:07.636445: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-29 06:48:09.808863: W tensorflow/compiler/

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("el_core_news_lg")

In [ ]:
df = pd.read_excel("/GHD.xlsx","Φύλλο1")

In [ ]:
def clean_data(x):
    puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#',
              '*', '+', '\\', '•', '~', '@', '£', '·', '_', '{', '}', '©', '^', '®', '`', '<', '→', '°', '€', '™', '›',
              '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…',
              '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―',
              '¥', '▓', '—', '‹', '─',
              '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸',
              '¾', 'Ã', '⋅', '‘', '∞',
              '∙', '）', '↓', '、', '│', '（', '»', '«', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø',
              '¹', '≤', '‡', '√', ]

    x = str(x)
    for punct in puncts:
        x = x.replace(punct, " ")

    x = x.lower()

    return x

In [ ]:
df["cleanText"] = df["text"].apply(lambda x: clean_data(x))

In [ ]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()

In [ ]:
df['cleanText'] = df['cleanText'].map(preprocess)

In [ ]:
df

,text,label,cleanText
0,Μέσα σε 27 μήνες εκδόθηκαν 570 χιλιάδες εκκρεμ...,0,μέσα σε 27 μήνες εκδόθηκαν 570 χιλιάδες εκκρεμ...
1,"«Δάνεια με επιτόκιο 0,35% για μικρές επιχειρήσ...",0,δάνεια με επιτόκιο 0 35 για μικρές επιχειρήσεις
2,ΟΙ 234 ΝΕΟΙ ΥΠΟΨΗΦΙΟΙ»,0,οι 234 νεοι υποψηφιοι
3,«ΤΑ ΠΑΙΔΙΑ ΤΗΣ ΜΕΓΑΛΗΣ ΦΥΓΗΣ»,0,τα παιδια της μεγαλης φυγης
4,««ΜΑΣΤΡΟΠΟΣ Ο ΑΝΗΨΙΟΣ Πορνείο ανηλίκων το Μαξί...,0,μαστροπος ο ανηψιος πορνείο ανηλίκων το μαξίμου
...,...,...,...
9998,"η μονη Δευτερα που αγαπησα στη ζωη μου,ηταν η ...",1,η μονη δευτερα που αγαπησα στη ζωη μου ηταν η ...
9999,Εχω μαύρο μάτι στο καράτε.,1,εχω μαύρο μάτι στο καράτε
10000,με τσίμπησε μέλισσα και μου κεντρισε το ενδιαφ...,1,με τσίμπησε μέλισσα και μου κεντρισε το ενδιαφ...
10001,Που θα μείνουμε; -Στα λόγια.,1,που θα μείνουμε στα λόγια


In [ ]:
df['vector'] = df['cleanText'].apply(lambda text: nlp(text).vector)

# **10-fold-cross validation technique**

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# **Reshape the X_train and X_test vectors to fit the models**

In [ ]:
X = df.vector.values
X_2d = np.stack(X)


# **scalling for negative value elimination**

**MinMaxScaler**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_2d)

 # **Spacy Word Embeddings on MultinomialNB Classifier**

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# prepare the cross-validation procedure
folds  = KFold(n_splits=10, random_state=42, shuffle=True)

# evaluate model
scores = cross_val_score(MultinomialNB(), scaled_train_embed, df.label, scoring='accuracy', cv=folds, n_jobs=-1)

for score in scores:
    print("Accuracy for this fold is: ", score)

# Mean accuracy
print(' Mean accuracy over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(MultinomialNB(), scaled_train_embed, df.label, scoring='f1', cv=folds, n_jobs=-1)

for score in scores:
    print("F1 for this fold is: ", score)

# Mean f1
print(' Mean F1 over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(MultinomialNB(), scaled_train_embed, df.label, scoring='recall', cv=folds, n_jobs=-1)

for score in scores:
    print("Recall for this fold is: ", score)

# Mean Recall
print(' Mean recall over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(MultinomialNB(), scaled_train_embed, df.label, scoring='precision', cv=folds, n_jobs=-1)

for score in scores:
    print("Precision for this fold is: ", score)

# Mean precision
print(' Mean precision over all folds is: ', (np.mean(scores)))

Accuracy for this fold is:  0.7552447552447552
Accuracy for this fold is:  0.7342657342657343
Accuracy for this fold is:  0.7652347652347652
Accuracy for this fold is:  0.734
Accuracy for this fold is:  0.759
Accuracy for this fold is:  0.713
Accuracy for this fold is:  0.737
Accuracy for this fold is:  0.76
Accuracy for this fold is:  0.73
Accuracy for this fold is:  0.722
 Mean accuracy over all folds is:  0.7409745254745255

F1 for this fold is:  0.7450572320499481
F1 for this fold is:  0.736111111111111
F1 for this fold is:  0.7638190954773869
F1 for this fold is:  0.7334669338677353
F1 for this fold is:  0.7582748244734202
F1 for this fold is:  0.7177974434611604
F1 for this fold is:  0.740375123395854
F1 for this fold is:  0.7678916827852998
F1 for this fold is:  0.731610337972167
F1 for this fold is:  0.7311411992263057
 Mean F1 over all folds is:  0.7425544983820389

Recall for this fold is:  0.7458333333333333
Recall for this fold is:  0.7540650406504065
Recall for this fold i

# **Spacy Word Embeddings BernoulliNB**

In [ ]:
from sklearn.naive_bayes import BernoulliNB


# prepare the cross-validation procedure
folds  = KFold(n_splits=10, random_state=42, shuffle=True)

# evaluate model
scores = cross_val_score(BernoulliNB(), scaled_train_embed, df.label, scoring='accuracy', cv=folds, n_jobs=-1)

for score in scores:
    print("Accuracy for this fold is: ", score)

# Mean accuracy
print(' Mean accuracy over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(BernoulliNB(),scaled_train_embed, df.label, scoring='f1', cv=folds, n_jobs=-1)

for score in scores:
    print("F1 for this fold is: ", score)

# Mean f1
print(' Mean F1 over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(BernoulliNB(binarize=1), scaled_train_embed, df.label, scoring='recall', cv=folds, n_jobs=-1)

for score in scores:
    print("Recall for this fold is: ", score)

# Mean Recall
print(' Mean recall over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(BernoulliNB(), scaled_train_embed, df.label, scoring='precision', cv=folds, n_jobs=-1)

for score in scores:
    print("Precision for this fold is: ", score)

# Mean precision
print(' Mean precision over all folds is: ', (np.mean(scores)))

Accuracy for this fold is:  0.47952047952047955
Accuracy for this fold is:  0.4935064935064935
Accuracy for this fold is:  0.4885114885114885
Accuracy for this fold is:  0.484
Accuracy for this fold is:  0.497
Accuracy for this fold is:  0.501
Accuracy for this fold is:  0.508
Accuracy for this fold is:  0.533
Accuracy for this fold is:  0.504
Accuracy for this fold is:  0.519
 Mean accuracy over all folds is:  0.5007538461538461

F1 for this fold is:  0.648210668467252
F1 for this fold is:  0.6599597585513078
F1 for this fold is:  0.6554508748317631
F1 for this fold is:  0.6518218623481781
F1 for this fold is:  0.6635451505016723
F1 for this fold is:  0.6675549633577615
F1 for this fold is:  0.6733067729083665
F1 for this fold is:  0.6953685583822571
F1 for this fold is:  0.669773635153129
F1 for this fold is:  0.6833443054641211
 Mean F1 over all folds is:  0.6668336549965808

Recall for this fold is:  1.0
Recall for this fold is:  1.0
Recall for this fold is:  1.0
Recall for this fo

# **Spacy Word Embeddings on Random Forest (RF)**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# prepare the cross-validation procedure
folds  = KFold(n_splits=10, random_state=42, shuffle=True)

# evaluate model
scores = cross_val_score(RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0),
                        scaled_train_embed, df.label, scoring='accuracy', cv=folds, n_jobs=-1)

for score in scores:
    print("Accuracy for this fold is: ", score)

# Mean accuracy
print(' Mean accuracy over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0),
                         scaled_train_embed, df.label, scoring='f1', cv=folds, n_jobs=-1)

for score in scores:
    print("F1 for this fold is: ", score)

# Mean f1
print(' Mean F1 over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0),
                         scaled_train_embed, df.label, scoring='recall', cv=folds, n_jobs=-1)

for score in scores:
    print("Recall for this fold is: ", score)

# Mean Recall
print(' Mean recall over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0),
                         scaled_train_embed, df.label, scoring='precision', cv=folds, n_jobs=-1)

for score in scores:
    print("Precision for this fold is: ", score)

# Mean precision
print(' Mean precision over all folds is: ', (np.mean(scores)))

Accuracy for this fold is:  0.8721278721278721
Accuracy for this fold is:  0.8621378621378621
Accuracy for this fold is:  0.8731268731268731
Accuracy for this fold is:  0.851
Accuracy for this fold is:  0.867
Accuracy for this fold is:  0.851
Accuracy for this fold is:  0.848
Accuracy for this fold is:  0.872
Accuracy for this fold is:  0.859
Accuracy for this fold is:  0.855
 Mean accuracy over all folds is:  0.8610392607392606

F1 for this fold is:  0.8693877551020408
F1 for this fold is:  0.8617234468937877
F1 for this fold is:  0.8738828202581926
F1 for this fold is:  0.8502512562814071
F1 for this fold is:  0.8676616915422886
F1 for this fold is:  0.8546341463414634
F1 for this fold is:  0.8532818532818534
F1 for this fold is:  0.8796992481203008
F1 for this fold is:  0.8645533141210375
F1 for this fold is:  0.8609779482262703
 Mean F1 over all folds is:  0.8636053480168643

Recall for this fold is:  0.8875
Recall for this fold is:  0.8739837398373984
Recall for this fold is:  0.9

# **Spacy Word Embeddings on SVM with rbf Kernel**

In [ ]:
from sklearn.svm import SVC

# prepare the cross-validation procedure
folds  = KFold(n_splits=10, random_state=42, shuffle=True)

# evaluate model
scores = cross_val_score(SVC(), scaled_train_embed, df.label, scoring='accuracy', cv=folds, n_jobs=-1)

for score in scores:
    print("Accuracy for this fold is: ", score)

# Mean accuracy
print(' Mean accuracy over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(SVC(), scaled_train_embed, df.label, scoring='f1', cv=folds, n_jobs=-1)

for score in scores:
    print("F1 for this fold is: ", score)

# Mean f1
print(' Mean F1 over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(SVC(), scaled_train_embed, df.label, scoring='recall', cv=folds, n_jobs=-1)

for score in scores:
    print("Recall for this fold is: ", score)

# Mean Recall
print(' Mean recall over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(SVC(), scaled_train_embed, df.label, scoring='precision', cv=folds, n_jobs=-1)

for score in scores:
    print("Precision for this fold is: ", score)

# Mean precision
print(' Mean precision over all folds is: ', (np.mean(scores)))

Accuracy for this fold is:  0.9310689310689311
Accuracy for this fold is:  0.9050949050949051
Accuracy for this fold is:  0.926073926073926
Accuracy for this fold is:  0.911
Accuracy for this fold is:  0.929
Accuracy for this fold is:  0.922
Accuracy for this fold is:  0.919
Accuracy for this fold is:  0.924
Accuracy for this fold is:  0.906
Accuracy for this fold is:  0.913
 Mean accuracy over all folds is:  0.9186237762237763

F1 for this fold is:  0.9283489096573209
F1 for this fold is:  0.9025641025641026
F1 for this fold is:  0.924643584521385
F1 for this fold is:  0.9077720207253887
F1 for this fold is:  0.9271794871794873
F1 for this fold is:  0.9219999999999999
F1 for this fold is:  0.9194029850746269
F1 for this fold is:  0.9280303030303031
F1 for this fold is:  0.906374501992032
F1 for this fold is:  0.9151219512195121
 Mean F1 over all folds is:  0.9181437845964158

Recall for this fold is:  0.93125
Recall for this fold is:  0.8943089430894309
Recall for this fold is:  0.932

# **Spacy Word Embeddings on SVM with linear Kernel**

In [ ]:
from sklearn.svm import SVC

# prepare the cross-validation procedure
folds  = KFold(n_splits=10, random_state=42, shuffle=True)

# evaluate model
scores = cross_val_score(SVC(kernel='linear', gamma='auto'),
                         scaled_train_embed, df.label, scoring='accuracy', cv=folds, n_jobs=-1)

for score in scores:
    print("Accuracy for this fold is: ", score)

# Mean accuracy
print(' Mean accuracy over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(SVC(kernel='linear', gamma='auto'),
                         scaled_train_embed, df.label, scoring='f1', cv=folds, n_jobs=-1)

for score in scores:
    print("F1 for this fold is: ", score)

# Mean f1
print(' Mean F1 over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(SVC(kernel='linear', gamma='auto'),
                         scaled_train_embed, df.label, scoring='recall', cv=folds, n_jobs=-1)

for score in scores:
    print("Recall for this fold is: ", score)

# Mean Recall
print(' Mean recall over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(SVC(kernel='linear', gamma='auto'),
                         scaled_train_embed, df.label, scoring='precision', cv=folds, n_jobs=-1)

for score in scores:
    print("Precision for this fold is: ", score)

# Mean precision
print(' Mean precision over all folds is: ', (np.mean(scores)))

Accuracy for this fold is:  0.8871128871128872
Accuracy for this fold is:  0.8631368631368631
Accuracy for this fold is:  0.903096903096903
Accuracy for this fold is:  0.877
Accuracy for this fold is:  0.895
Accuracy for this fold is:  0.892
Accuracy for this fold is:  0.88
Accuracy for this fold is:  0.892
Accuracy for this fold is:  0.883
Accuracy for this fold is:  0.879
 Mean accuracy over all folds is:  0.8851346653346652

F1 for this fold is:  0.8848114169215088
F1 for this fold is:  0.8623115577889447
F1 for this fold is:  0.903096903096903
F1 for this fold is:  0.875882946518668
F1 for this fold is:  0.8953140578265204
F1 for this fold is:  0.8934911242603552
F1 for this fold is:  0.8832684824902725
F1 for this fold is:  0.897920604914934
F1 for this fold is:  0.8858536585365854
F1 for this fold is:  0.885308056872038
 Mean F1 over all folds is:  0.886725880922673

Recall for this fold is:  0.9041666666666667
Recall for this fold is:  0.8719512195121951
Recall for this fold is:

# **Spacy Word Embeddings on KNeighbors Classifier**

In [ ]:
from  sklearn.neighbors import KNeighborsClassifier

# prepare the cross-validation procedure
folds  = KFold(n_splits=10, random_state=42, shuffle=True)

# evaluate model
scores = cross_val_score(KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean'),
                         scaled_train_embed, df.label, scoring='accuracy', cv=folds, n_jobs=-1)

for score in scores:
    print("Accuracy for this fold is: ", score)

# Mean accuracy
print(' Mean accuracy over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean'),
                         scaled_train_embed, df.label, scoring='f1', cv=folds, n_jobs=-1)

for score in scores:
    print("F1 for this fold is: ", score)

# Mean f1
print(' Mean F1 over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean'),
                         scaled_train_embed, df.label, scoring='recall', cv=folds, n_jobs=-1)

for score in scores:
    print("Recall for this fold is: ", score)

# Mean Recall
print(' Mean recall over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean'),
                         scaled_train_embed, df.label, scoring='precision', cv=folds, n_jobs=-1)

for score in scores:
    print("Precision for this fold is: ", score)

# Mean precision
print(' Mean precision over all folds is: ', (np.mean(scores)))

Accuracy for this fold is:  0.8541458541458542
Accuracy for this fold is:  0.8271728271728271
Accuracy for this fold is:  0.8551448551448552
Accuracy for this fold is:  0.82
Accuracy for this fold is:  0.837
Accuracy for this fold is:  0.839
Accuracy for this fold is:  0.84
Accuracy for this fold is:  0.847
Accuracy for this fold is:  0.815
Accuracy for this fold is:  0.83
 Mean accuracy over all folds is:  0.8364463536463536

F1 for this fold is:  0.8612167300380228
F1 for this fold is:  0.8402585410895661
F1 for this fold is:  0.8648648648648648
F1 for this fold is:  0.8320895522388061
F1 for this fold is:  0.8511415525114155
F1 for this fold is:  0.8535031847133757
F1 for this fold is:  0.85663082437276
F1 for this fold is:  0.8659070990359334
F1 for this fold is:  0.8337825696316262
F1 for this fold is:  0.8482142857142858
 Mean F1 over all folds is:  0.8507609204210658

Recall for this fold is:  0.94375
Recall for this fold is:  0.9247967479674797
Recall for this fold is:  0.95277

# **Spacy Word Embeddings on Gradient Boosted Decision Trees (GBDT)**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# prepare the cross-validation procedure
folds  = KFold(n_splits=10, random_state=42, shuffle=True)

# evaluate model
scores = cross_val_score(GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0),
                         scaled_train_embed, df.label, scoring='accuracy', cv=folds, n_jobs=-1)

for score in scores:
    print("Accuracy for this fold is: ", score)

# Mean accuracy
print(' Mean accuracy over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0),
                         scaled_train_embed, df.label, scoring='f1', cv=folds, n_jobs=-1)

for score in scores:
    print("F1 for this fold is: ", score)

# Mean f1
print(' Mean F1 over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0),
                         scaled_train_embed, df.label, scoring='recall', cv=folds, n_jobs=-1)

for score in scores:
    print("Recall for this fold is: ", score)

# Mean Recall
print(' Mean recall over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0),
                         scaled_train_embed, df.label, scoring='precision', cv=folds, n_jobs=-1)

for score in scores:
    print("Precision for this fold is: ", score)

# Mean precision
print(' Mean precision over all folds is: ', (np.mean(scores)))

Accuracy for this fold is:  0.8711288711288712
Accuracy for this fold is:  0.8621378621378621
Accuracy for this fold is:  0.8861138861138861
Accuracy for this fold is:  0.859
Accuracy for this fold is:  0.86
Accuracy for this fold is:  0.859
Accuracy for this fold is:  0.861
Accuracy for this fold is:  0.889
Accuracy for this fold is:  0.853
Accuracy for this fold is:  0.863
 Mean accuracy over all folds is:  0.8663380619380618

F1 for this fold is:  0.8685015290519879
F1 for this fold is:  0.8603238866396761
F1 for this fold is:  0.8855421686746987
F1 for this fold is:  0.8556806550665302
F1 for this fold is:  0.8605577689243028
F1 for this fold is:  0.8605341246290801
F1 for this fold is:  0.8646543330087633
F1 for this fold is:  0.8959700093720712
F1 for this fold is:  0.855457227138643
F1 for this fold is:  0.866601752677702
 Mean F1 over all folds is:  0.8673823455183456

Recall for this fold is:  0.8875
Recall for this fold is:  0.8638211382113821
Recall for this fold is:  0.9055

# **Spacy Word Embeddings on Stochastic Gradient Descent (SGDC)**

In [ ]:
from sklearn.linear_model import SGDClassifier

# prepare the cross-validation procedure
folds  = KFold(n_splits=10, random_state=42, shuffle=True)

# evaluate model
scores = cross_val_score(SGDClassifier(loss="hinge", penalty="l2", max_iter=100),
                        scaled_train_embed, df.label, scoring='accuracy', cv=folds, n_jobs=-1)

for score in scores:
    print("Accuracy for this fold is: ", score)

# Mean accuracy
print(' Mean accuracy over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(SGDClassifier(loss="hinge", penalty="l2", max_iter=100),
                         scaled_train_embed, df.label, scoring='f1', cv=folds, n_jobs=-1)

for score in scores:
    print("F1 for this fold is: ", score)

# Mean f1
print(' Mean F1 over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(SGDClassifier(loss="hinge", penalty="l2", max_iter=100),
                         scaled_train_embed, df.label, scoring='recall', cv=folds, n_jobs=-1)

for score in scores:
    print("Recall for this fold is: ", score)

# Mean Recall
print(' Mean recall over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(SGDClassifier(loss="hinge", penalty="l2", max_iter=100),
                         scaled_train_embed, df.label, scoring='precision', cv=folds, n_jobs=-1)

for score in scores:
    print("Precision for this fold is: ", score)

# Mean precision
print(' Mean precision over all folds is: ', (np.mean(scores)))

Accuracy for this fold is:  0.8741258741258742
Accuracy for this fold is:  0.8581418581418582
Accuracy for this fold is:  0.8791208791208791
Accuracy for this fold is:  0.873
Accuracy for this fold is:  0.895
Accuracy for this fold is:  0.872
Accuracy for this fold is:  0.851
Accuracy for this fold is:  0.87
Accuracy for this fold is:  0.863
Accuracy for this fold is:  0.875
 Mean accuracy over all folds is:  0.8710388611388611

F1 for this fold is:  0.8444846292947558
F1 for this fold is:  0.8459869848156182
F1 for this fold is:  0.8819047619047619
F1 for this fold is:  0.8767676767676769
F1 for this fold is:  0.8384879725085911
F1 for this fold is:  0.8918406072106263
F1 for this fold is:  0.8425531914893617
F1 for this fold is:  0.8362877997914494
F1 for this fold is:  0.7898966704936854
F1 for this fold is:  0.882998171846435
 Mean F1 over all folds is:  0.8531208466122961

Recall for this fold is:  0.7729166666666667
Recall for this fold is:  0.9207317073170732
Recall for this fol

# **Spacy Word Embeddings on Decision Trees (DT)**

In [ ]:
from sklearn import tree


# prepare the cross-validation procedure
folds  = KFold(n_splits=10, random_state=42, shuffle=True)

# evaluate model
scores = cross_val_score(tree.DecisionTreeClassifier(max_depth=30, random_state=0),
                         scaled_train_embed, df.label, scoring='accuracy', cv=folds, n_jobs=-1)

for score in scores:
    print("Accuracy for this fold is: ", score)

# Mean accuracy
print(' Mean accuracy over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(tree.DecisionTreeClassifier(max_depth=30, random_state=0),
                         scaled_train_embed, df.label, scoring='f1', cv=folds, n_jobs=-1)

for score in scores:
    print("F1 for this fold is: ", score)

# Mean f1
print(' Mean F1 over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(tree.DecisionTreeClassifier(max_depth=30, random_state=0),
                         scaled_train_embed, df.label, scoring='recall', cv=folds, n_jobs=-1)

for score in scores:
    print("Recall for this fold is: ", score)

# Mean Recall
print(' Mean recall over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(tree.DecisionTreeClassifier(max_depth=30, random_state=0),
                         scaled_train_embed, df.label, scoring='precision', cv=folds, n_jobs=-1)

for score in scores:
    print("Precision for this fold is: ", score)

# Mean precision
print(' Mean precision over all folds is: ', (np.mean(scores)))

Accuracy for this fold is:  0.7832167832167832
Accuracy for this fold is:  0.7712287712287712
Accuracy for this fold is:  0.8081918081918081
Accuracy for this fold is:  0.776
Accuracy for this fold is:  0.795
Accuracy for this fold is:  0.82
Accuracy for this fold is:  0.78
Accuracy for this fold is:  0.768
Accuracy for this fold is:  0.76
Accuracy for this fold is:  0.769
 Mean accuracy over all folds is:  0.7830637362637363

F1 for this fold is:  0.7755946225439503
F1 for this fold is:  0.7698492462311558
F1 for this fold is:  0.8076152304609218
F1 for this fold is:  0.7700205338809035
F1 for this fold is:  0.7952047952047953
F1 for this fold is:  0.8228346456692913
F1 for this fold is:  0.7843137254901961
F1 for this fold is:  0.780718336483932
F1 for this fold is:  0.7665369649805448
F1 for this fold is:  0.7755102040816327
 Mean F1 over all folds is:  0.7848198305027324

Recall for this fold is:  0.78125
Recall for this fold is:  0.7784552845528455
Recall for this fold is:  0.8275

# **Spacy Word Embeddings on Logistic Regression (LR)**

In [ ]:
from sklearn.linear_model import LogisticRegression

# prepare the cross-validation procedure
folds  = KFold(n_splits=10, random_state=42, shuffle=True)

# evaluate model
scores = cross_val_score(LogisticRegression(solver='liblinear', multi_class='ovr', random_state=0),
                         scaled_train_embed, df.label, scoring='accuracy', cv=folds, n_jobs=-1)

for score in scores:
    print("Accuracy for this fold is: ", score)

# Mean accuracy
print(' Mean accuracy over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(LogisticRegression(solver='liblinear', multi_class='ovr', random_state=0),
                         scaled_train_embed, df.label, scoring='f1', cv=folds, n_jobs=-1)

for score in scores:
    print("F1 for this fold is: ", score)

# Mean f1
print(' Mean F1 over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(LogisticRegression(solver='liblinear', multi_class='ovr', random_state=0),
                         scaled_train_embed, df.label, scoring='recall', cv=folds, n_jobs=-1)

for score in scores:
    print("Recall for this fold is: ", score)

# Mean Recall
print(' Mean recall over all folds is: ', (np.mean(scores)))
print()

# evaluate model
scores = cross_val_score(LogisticRegression(solver='liblinear', multi_class='ovr', random_state=0),
                         scaled_train_embed, df.label, scoring='precision', cv=folds, n_jobs=-1)

for score in scores:
    print("Precision for this fold is: ", score)

# Mean precision
print(' Mean precision over all folds is: ', (np.mean(scores)))

Accuracy for this fold is:  0.8791208791208791
Accuracy for this fold is:  0.8691308691308691
Accuracy for this fold is:  0.8991008991008991
Accuracy for this fold is:  0.876
Accuracy for this fold is:  0.894
Accuracy for this fold is:  0.889
Accuracy for this fold is:  0.875
Accuracy for this fold is:  0.891
Accuracy for this fold is:  0.871
Accuracy for this fold is:  0.879
 Mean accuracy over all folds is:  0.8822352647352647

F1 for this fold is:  0.8771573604060914
F1 for this fold is:  0.8670050761421318
F1 for this fold is:  0.8986960882647944
F1 for this fold is:  0.8739837398373984
F1 for this fold is:  0.8944223107569721
F1 for this fold is:  0.8908554572271388
F1 for this fold is:  0.8785228377065113
F1 for this fold is:  0.8972667295004711
F1 for this fold is:  0.8741463414634147
F1 for this fold is:  0.8857412653446647
 Mean F1 over all folds is:  0.8837797206649588

Recall for this fold is:  0.9
Recall for this fold is:  0.8678861788617886
Recall for this fold is:  0.9199